[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ozgurural/SecurePoL-with-Watermarking/blob/main/notebooks/analysis_notebook.ipynb)

# 📊 SecurePoL-with-Watermarking — *Analysis Notebook*
This notebook rebuilds **every** table & figure appearing in the dissertation from the four Colab training runs stored in Google Drive.

In [ ]:
# 🔧 environment & Google Drive
from google.colab import drive
drive.mount('/content/drive')

!pip -q install pandas matplotlib seaborn tqdm

ROOT = '/content/drive/MyDrive/SecurePoL-with-Watermarking'  # ← change if needed

## 1 Locate experiment folders

In [ ]:
from pathlib import Path

runs = {
    'baseline'      : 'CIFAR10_Run',
    'feature_based' : 'CIFAR10_feature_based',
    'non_intrusive' : 'CIFAR10_non_intrusive',
    'param_pert'    : 'CIFAR10_param_pert',
}
proof_root = Path(ROOT) / 'proof'
for tag, folder in runs.items():
    assert (proof_root / folder).exists(), f"❌ folder {folder} missing"
runs

## 2 Build master KPI table

In [ ]:
import json, re, pandas as pd

rows = []
for tag, folder in runs.items():
    hist = json.load(open(proof_root / folder / 'metrics.json'))
    last = hist[-1]
    wall = last.get('wall_time')
    if wall is None:
        txt = (proof_root / folder / 'train.log').read_text()
        m = re.search(r'Total wall-clock time:\s+([\d.]+)s', txt)
        wall = float(m.group(1)) if m else None
    rows.append(dict(setup=tag,
                     val_acc=last['val_acc'],
                     val_loss=last['val_loss'],
                     wall_sec=wall))

df = pd.DataFrame(rows).set_index('setup')
df

In [ ]:
Path('/content/figs').mkdir(exist_ok=True)
df.to_csv('/content/figs/table_kpi.csv')
df.to_markdown('/content/figs/table_kpi.md')

## 3 Training / validation curves

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,4))
for tag, folder in runs.items():
    hist = json.load(open(proof_root / folder / 'metrics.json'))
    plt.plot([e['val_acc'] for e in hist], label=tag.replace('_',' '))
plt.xlabel('epoch'); plt.ylabel('val accuracy (%)'); plt.legend(); plt.grid()
plt.tight_layout(); plt.savefig('/content/figs/curve_acc.png', dpi=200)
plt.show()

plt.figure(figsize=(7,4))
for tag, folder in runs.items():
    hist = json.load(open(proof_root / folder / 'metrics.json'))
    plt.plot([e['val_loss'] for e in hist], label=tag.replace('_',' '))
plt.xlabel('epoch'); plt.ylabel('val loss'); plt.legend(); plt.grid()
plt.tight_layout(); plt.savefig('/content/figs/curve_loss.png', dpi=200)
plt.show()

## 4 Runtime / accuracy overhead

In [ ]:
base_acc = df.loc['baseline','val_acc']
base_t   = df.loc['baseline','wall_sec']

df['dAcc_pp'] = df['val_acc'] - base_acc
df['dTime_s'] = df['wall_sec'] - base_t

fig, ax = plt.subplots(1,2, figsize=(9,3))
df['dAcc_pp'].plot(kind='bar', ax=ax[0]); ax[0].set_ylabel('Δ accuracy (pp)')
df['dTime_s'].plot(kind='bar', ax=ax[1], color='orange'); ax[1].set_ylabel('Δ time (s)')
for a in ax: a.grid(True, ls='--')
plt.suptitle('Overhead vs baseline');
plt.tight_layout(); plt.savefig('/content/figs/overhead.png', dpi=200)

## 5 PoL top-Q distance distribution

In [ ]:
import seaborn as sns, glob
records = []
for tag, folder in runs.items():
    csv = proof_root / folder / 'verify_full_metrics.csv'
    if csv.exists():
        vdf = pd.read_csv(csv)
        for m in ['1','2','inf','cos']:
            records += [dict(setup=tag, metric=m, value=v) for v in vdf[m]]
big = pd.DataFrame(records)

plt.figure(figsize=(6,4))
sns.boxplot(data=big, x='metric', y='value', hue='setup')
plt.yscale('log'); plt.grid(ls='--')
plt.title('PoL top-Q distance'); plt.tight_layout();
plt.savefig('/content/figs/pol_dist.png', dpi=200)

## 6 Watermark detectability over time (NI)

In [ ]:
# 🔍 Non-intrusive WM MSE vs checkpoint
import torch, numpy as np, glob
from watermark_utils import WatermarkModule, generate_trigger_inputs

ckpts = sorted(glob.glob(str(proof_root / runs['non_intrusive'] / 'model_step_*')))
mses  = []
for p in ckpts:
    st = torch.load(p, map_location='cpu')['net']
    wm = WatermarkModule(torch.nn.Identity(), 'secret_key', 128)  # dummy base
    wm.load_state_dict(st)
    x = generate_trigger_inputs('secret_key')
    with torch.no_grad():
        y = wm(x, trigger=True)
    mse = torch.nn.functional.mse_loss(y, torch.zeros_like(y)).item()
    mses.append(mse)

plt.figure(figsize=(6,3))
plt.plot(mses); plt.xlabel('checkpoint #'); plt.ylabel('WM MSE'); plt.grid()
plt.title('NI watermark strength across training')
plt.tight_layout(); plt.savefig('/content/figs/ni_mse.png', dpi=200)
plt.show()

## 7 Export everything for LaTeX / Overleaf

In [ ]:
# 💾 Save KPI table (Markdown) already done; ensure figs directory populated
!ls -lh /content/figs | head
print('✔ Artifacts ready in  /content/figs  (auto-sync with Drive)')

> **Done!** Import `/content/figs/*` into Overleaf and `table_kpi.md` into any Markdown-to-LaTeX tool. All graphs in the dissertation now reproduce with one click. 🚀